# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
#Read data and make dataframe
data = "../output_data/my_weather_data.csv"
weather_data = pd.read_csv(data)
weather_data
#weather_df = pd.DataFrame(weather_data)
#weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,Arlit,0,NE,1587265200,12,81.39,18.7369,7.3853,7.20
1,1,Naze,100,JP,1587265200,75,74.79,28.3667,129.4833,15.79
2,2,Barrow,57,US,1587265200,94,8.60,71.2906,-156.7887,7.58
3,3,Padang,16,ID,1587265200,66,86.52,-0.9492,100.3543,1.05
4,4,Port Macquarie,9,AU,1587265200,65,73.02,-31.4333,152.9167,5.70
...,...,...,...,...,...,...,...,...,...,...
583,583,Kyshtovka,100,RU,1587265200,64,50.63,56.5550,76.6271,10.65
584,584,Yaqui,0,MX,1587265200,48,70.77,27.3667,-110.1667,6.06
585,585,Nur-Sultan,98,KZ,1587265200,47,55.54,51.1801,71.4460,1.45
586,586,Thinadhoo,75,MV,1587265200,76,83.91,0.5333,72.9333,11.65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
# Configure gmaps with API
gmaps.configure(api_key=g_key)

In [53]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

weather_df = weather_df.dropna()

humidity_info = weather_df["Humidity"].astype(float)

In [54]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_info, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

print(fig)


Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
#A max temperature lower than 80 degrees but higher than 70.
first_cond = weather_df.loc[weather_df["Temp"]<80,:]

second_cond = first_cond.loc[weather_df["Temp"]>70,:]
#second_cond.head()

#Less than 10mph Wind
third_cond = second_cond.loc[second_cond["Wind Speed"]<10,:]

#Filtering to zero cloudiness.
fourth_cond = third_cond.loc[third_cond["Cloudiness"]==0,:]
fourth_cond.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
71,71,Pisco,0,PE,1587265200,75,70.68,-13.7000,-76.2167,6.15
132,132,Rock Sound,0,BS,1587265200,76,78.21,24.9000,-76.2000,5.35
266,266,Dogondoutchi,0,NE,1587265200,14,79.93,13.6393,4.0287,3.69
270,270,Morondava,0,MG,1587265200,79,74.55,-20.2833,44.2833,7.58
485,485,Sillanwali,0,PK,1587265200,46,76.96,31.8261,72.5398,5.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth_cond = pd.DataFrame(fourth_cond)
hotel_df['City'] = fifth_cond['City']
hotel_df['Country'] = fifth_cond['Country']
hotel_df['Lat'] = fifth_cond['Lat']
hotel_df['Lng'] = fifth_cond['Lng']
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
71,Pisco,PE,-13.7000,-76.2167,
132,Rock Sound,BS,24.9000,-76.2000,
266,Dogondoutchi,NE,13.6393,4.0287,
270,Morondava,MG,-20.2833,44.2833,
485,Sillanwali,PK,31.8261,72.5398,


In [57]:
# use iterrows to scan dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={lat},{lng}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {lat} , {lng} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing Information")
        
    print("------------")

Retrieving Results for Index 71: Pisco.
Closest hotel to Pisco at -13.7 , -76.2167 is Posada Gino.
------------
Retrieving Results for Index 132: Rock Sound.
Closest hotel to Rock Sound at 24.9 , -76.2 is Centre for Training and Innovation (CTI).
------------
Retrieving Results for Index 266: Dogondoutchi.
Closest hotel to Dogondoutchi at 13.6393 , 4.0287 is Public School.
------------
Retrieving Results for Index 270: Morondava.
Closest hotel to Morondava at -20.2833 , 44.2833 is School De L'alliance Morondava.
------------
Retrieving Results for Index 485: Sillanwali.
Closest hotel to Sillanwali at 31.8261 , 72.5398 is esahulat 64205.
------------
Retrieving Results for Index 571: Mayahi.
Missing Information
------------
Retrieving Results for Index 584: Yaqui.
Closest hotel to Yaqui at 27.3667 , -110.1667 is telesecundaria # 13.
------------


In [58]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
71,Pisco,PE,-13.7000,-76.2167,Posada Gino
132,Rock Sound,BS,24.9000,-76.2000,Centre for Training and Innovation (CTI)
266,Dogondoutchi,NE,13.6393,4.0287,Public School
270,Morondava,MG,-20.2833,44.2833,School De L'alliance Morondava
485,Sillanwali,PK,31.8261,72.5398,esahulat 64205


In [60]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity_info, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

marks = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
#fig.add_layer(marks)

# Display Map
fig

ValueError: weights must be of the same length as locations or None